# Tutorial
This is a short tutorial to get started with Bayesian Networks and the BayesNet library.

## Random variables and probability distributions

The real world is uncertain ... weather, stock prices, sensor readings etc. A mathematically sound way to deal with uncertainty is to use **probabilities**, e.g. the probability that it will rain on a random day:
$$ P(Rain=true)=0.16 $$
"Rain" is a **discrete random variable** that can have values $\{false, true\}$

Probabilities add to 1 so:
$$ P(Rain=false) = 1 - P(Rain=true) = 0.84 $$
The notation $P(Rain)$ (without a value for the variable) denotes a vector containing the probabilites for all possible values of Rain:
$$ p(Rain) = \begin{pmatrix} p(Rain=false) \\ p(Rain=true) \end{pmatrix} = \begin{pmatrix} 0.84 \\ 0.16 \end{pmatrix} $$

$P(Rain)$ is a **discrete probability distribution** over all possible values of Rain. It can also be represented as a **probability table**. The rows in a probability table must sum to 1:

![](pd_rain.svg)

In **graphical models**, like **Bayesian networks**, random variables are represented as cirles:

![](one_variable.svg)

Things get interesting, when you have more than one variable in your model. Then, it can happen, that **one variable influences another variable**.
The graphical way to represent this relation between the two variables is an arrow, e.g. the season influences the probability for rain:

![](season_rain.svg)

This could be read as:
* season **influences** or **causes** rain or
* rain **depends on** or **is influenced by** season

Assume that "Season" has two values $ \{winter, summer\} $, then the probability that it rains is higher, when we know it is winter, than when it is summer.  
This is now a **conditional probability**: 
$$P(Rain|Season)$$

The bar "|" is called the "conditioning bar" and reads as "given": the probabilty of "Rain" given the "Season". Knowing the season, would could change our belief, whether it would rain today.

We can express this as a table:

![](cpd_rain_given_season.svg)

This is a **conditional probability table (CPD)**. The first line is $P(Rain|Season=winter)$ and the second line is $P(Rain|Season=summer)$

Note that the values in the rows of the table sum to one. Each row represents a (conditional) probability distribution.

## Chain Rule for Bayesian Networks

Now, the **key to understand Bayesian Networks** is, that the diagram above with the two random variables "Season" and "Rain" represents a joint probability. It is the joint probability of season and rain: $ P(Season,Rain)$.

![](joint_season_rain.svg)

And there is a simple rule to calculate a joint probability corresponding to Bayesian Netorks. It the called **Chain Rule for Bayesian Netorks**. 
To get the joint probaility given by a diagram, you simply build a product of factors. You have a factor for each variable. And the factors are the probability distributions of the variables. For variables without incoming edges, the factor is the marginal probability, e.g. $P(Season)$, and for variables with incoming edges, the factor is a conditional probability, e.g. $P(Rain|Season)$

So in the case above, $P(Season, Rain)$ factorizes in:

$$ P(Season,Rain) = P(Season) * P(Rain|Season) $$

Variables that we condition on, like "Season", are called "parents" (pa). "Season" is a parent of "Rain". The general formula for the Chain rule for Bayesian Networks is:

$$ P(X_1, X_2, ... , X_n) = \prod_i P(X_i|pa(X_i))$$
<br>
<div class="alert alert-block alert-warning"><b>Note:</b> A Bayesian network is just a graphical representation of a joint probability distribution. The arrows in the diagram indicate, how this joint distribution factorizes into (conditional) probabilities</div>


## Inference in Bayesian Networks

You might say: **"What is the benefit of knowing the joint distribution and how it factorizes?"**

The benefit of knowing the joint distribution and its factorization is, that we can derive (infer) values from it, that are of interest and are not given.

For example, from the above joint and factorization of "Season" and "Rain" $P(Season,Rain) = P(Season) * P(Rain|Season)$ we could compute the following distributions:
* $P(Season|Rain)$: the probability of the season, given it is raining
* $P(Rain)$: the (unconditional/marginal) probability of rain

Calculating such values is called **inference**.

You can think of a joint distribution as a database, that you can query for information. The information in this "database" is linked like in a brain. And when you query the database, these links are considered when calculating the result.


## Factors

When doing calculations based on Bayesian Networks, we often do not calculate with probabilities, but with "factors". **Factors** are similar to probability distributions, but more general:
<br>
<div class="alert alert-block alert-warning"><b>Factor:</b> A factor F is simply a function that maps a certain configuration (aka. assignments) of its arguments to a real value</div>

$$ F(arg_1,arg_2, ... , arg_n) \mapsto \mathbb{R} $$

If the arguments are discrete values, then we can represent a factor, similar to a (conditional) probability, as a table.
But in contrast to conditional probability tables (CPTs), where the probability values are stored in a matrix, the values in factors are contained in a "flat" vector. For the values of the variables, like $Season \in \{winter,summer\}$ integer values are used $Season \in \{0,1\}$.

Since factors are more general functions than probability distributions, their values don't have to sum to 1 and the values can even be negative.

We can convert probability distributions in factors. In the diagram you see the probability distributions of "Season" and "Rain" on the left and their corresponding factors on the right.

![](cpds_and_factors.svg)

<br>
<div class="alert alert-block alert-warning"><b>Note:</b> In the BayesNet library, we follow the convention that the values of the first variable in the factor, which is in the first column, change most frequently.</div>

A factor has 3 attributes:

1. var:  list of variables (arguments) of the factor, aka. scope
2. card: list of the cardinalities of the variables in var
3. val:  list of values

Here is how you define a probability, like $P(Rain)$ and a CPD, like $P(Rain|Season)$ as factors in the BayesNet library:

```
Factor pd_rain{ {0}, {2}, { 0.86, 0.14 } };

Factor cpd_rain_given_season{ {1,0}, {2,2}, {0.82,0.18,0.86,0.14} };
```

Variables in BayesNet are identified with an unique id. The ids start with 0 and are contiguous. You can make the code more readable, if you define an enum for the variables:

```
enum VarIds : uint32_t {
	SEASON,
	RAIN
};

Factor pd_rain{ {RAIN}, {2},   { 0.86, 0.14 };

Factor cpd_rain_given_season{ {RAIN,SEASON}, {2,2}, {0.82,0.18,0.86,0.14} };
```

The set of variables var $X_1 ... X_n$ is called the **scope** of the factor. That is the set of arguments that a factor takes. A concrete configuration of the arguments is called **assignment**. By providing an assignment, you can lookup the corresponding value in the vector of values.

For example, in case of the factor rain_given_season

![](factor_rain_season.svg)

with

```
const auto value = cpd_rain_given_season({0,1});
```

you would get the value 0.86 from the 3rd line in the vector.

Again, you can make the code more readable using enums for the values of the discrete variables.

```
enum RainValues : uint32_t {
	NO,
	YES
};

enum SeasnonValues : uint32_t {
	WINTER,
	SUMMER
};

const auto value = cpd_rain_given_season({NO,SUMMER});
```

**How to build you own factors?**

When you want to define a new factor in the code, it is convenient to first create a factor table on paper with columns for all variables in the scope and one column for the factor values. Then add a row for all possible combinations of the (discrete) variable values. The number of possible combinations is the product of all cardinalities. So if you have 2 variables with cardinalities $\{2,2\}$, you get $2*2=4$ rows.

![](build_factor_rain_season1.svg)

In the columns for the variables, you write all possible combinations of discrete values of the variables. Start with the first column and repeat enumerating all values for the first variable until you reach the last row. 

![](build_factor_rain_season2.svg)

For the second variable/column, you also enumerate the values, but now with a slower frequency: for one enumeration of the variable to the left, you take one value, then for the next enumeration of the left variable, you take the next value.

![](build_factor_rain_season3.svg)

Then repeat for other variables until all variable columns are filled with all possible combinations. 

Then enter the factor values in the last column.

![](build_factor_rain_season4.svg)

<br>
<div class="alert alert-block alert-warning"><b>Note:</b> When working with factors, there is no notion of direction. From f(Season,Rain) we do not know that "Season" has an influence on "Rain". So without knowing the original graph with the directed edges, we cannot convert a factor back to a conditional probability distribution.</div>

Why would we use factors instead of probabilites for calculating values, when we loose information using factors? The reason is, that factor calculation can be used to calculate values also in other kind of graphical models like [Markov Random Fields](https://en.wikipedia.org/wiki/Markov_random_field) (undirected graphical model) and [Influence diagrams](https://en.wikipedia.org/wiki/Influence_diagram) (decision problems). 

![](MRF_ID.svg)

On the left side is a Markov Random field with undirected edges. On the right side is an Influence Diagram with special nodes for decision factors (rectangles) and utility functions (diamonds).

So "factor calculation" is an unifiying approach. Furthermore, often, we are only interested in calculating marginals, like $P(Rain)$, so there is no need to create a CPD from a factor. Lastly, when you know the original Bayes Net with the arrows, you can convert the factors to CPDs.

# Factor operations

After instantiating the factors, you can call operations defined for factors, like FactorProduct, FactorSum, Marginalize, ObserveEvidence (factor reduction) or Normalize. These are the building blocks for algorithms like VariableElimination or JunctionTree.

## FactorProduct
The FactorProduct multiplies two factors. If the factors represent probabilites or conditional probabilites, then the product will be the joint distribution.

Example:

![](factor_product.svg)

The FactorProduct is defined as a pointwise multiplication. For instance, for the two factors in the diagram above, the first row of the first factor has assignment Season=0 and value=0.5. The value 0.5 is then multiplied with all values of the second factor, that have Season=0, which are the first two rows.

The general rule is, that you take a row of the first factor and you multiply the value of this row with the values of the rows of the second factor, for which the assignement of the first factor matches the assignment of the second factor.

In BayesNet, the variables in the result of a factor operation are ordered. So in the resulting factor for the joint distribution (most right factor in the above diagram), Season with id 0 is in the first column and Rain with id 1 is in the second column. Notice that reordering the variables in a factor changes also the order of the values in the factor. In the example, reordering of the variables of the result swaps row 2 and 3.

```
Factor pd_season{ {SEASON}, {2}, { 0.5, 0.5 } };
Factor cpd_rain_given_season{ {RAIN,SEASON}, {2,2}, {0.82,0.18,0.86,0.14} };
Factor joint_season_rain = FactorProduct(season, rain_given_season);
```
Result:
```
joint_season_rain:
var=[0,1]
card=[2,2]
val=[0.41,0.43,0.09,0.07]
```

## FactorSum
FactorSum is like FactorProduct, except that corresponding values in the two factors are summed instead of multiplied.

## Marginalization
Marginalization removes a variable or multiple variables from the scope of a factor:
$$ f(Season, Rain) \to marginalize\ Season \to f(Rain) $$

![](factor_marginalization.svg)

Marginalization is done by removing the column(s) of the variable(s) to be removed and adding all values with the same remaining assignment. When we remove the column Season, there are two entries for Rain=0 and two entries for Rain=1 that get added.

```
Factor m_rain = joint_season_rain.Marginalize({SEASON});
```
Result:
```
m_rain:
var=[1]
card=[2]
val=[0.84,0.16]
```

## Observe evidence

If the values of some variables are observed, we can incorporate this knowledge into a factor or a list of factors.
Incorporating evidence in a factor is done by setting all values in the factor to zero, that are not consistent with the evidence.
In the example, we observe (i.e. know for sure), that it is summer (SEASON=TRUE), so all values in the factor that have SEASON=FALSE are set to 0.

![](observe_evidence.svg)

"Evidence" in BayeseNet is a list of variable/value pairs for the observed variables.
```
const Evidence evidence{ {SEASON, SUMMER} };
cpd_rain_given_season.ObserveEvidence(evidence);
```
gives
```
cpd_rain_given_season(Season=summer):
var=[1,0]
card=[2,2]
val=[0,0,0.86,0.14]
```
After incorporating the evidence, you can marginalize out the observed variables, because they are constants that will not change.
ObserveEvidence has also a boolean argument "marginalize", with which you can indicate, that the observed variables should be removed from the scope of the factor.
```
bool marginalize{ true };
cpd_rain_given_season.ObserveEvidence(evidence, marginalize);
```
This gives:
```
cpd_rain_given_season(Season=summer):
var=[1]
card=[2]
val=[0.86,0.14]
```

## Normalize
Normalize() normalizes the values of a factor to sum to 1. This is useful, if the result of a factor calculation is a marginal probability and you want that the values sum to 1.  

# Inference algorithms

Now you know how to calculate values of interest in Bayesian Networks with factor operations like FactorProduct, Marginalize and ObserveEvidence.

Up to now, we had only small networks, with only one or two variables. At the beginning, we started with a single random variable Rain, specifying the probability distribution for rain on a random day. Then we refined the model by adding a dependency to the season, because we stated that it rains more in winter than in summer. We can continue to refine the model more and more. Let's add a dependency to the country. In some countries it rains more than in others. We can also create a dependency between Season and Country, because in some countries, like Finland, the summer is short and the winter long, whereas in Egypt, it is vice versa.

![](country_season_rain.svg)

The code for the right diagram would be:

```
   // let's add another variable country
   // add additional id 
   const uint32_t COUNTRY{ 2 };
   // country has two values {false, true} which mean {Finland, Egypt}
   // the numbers in the factor represent the number of inhabitants in millions
   // if the factor should represent a probability, we have to normalize the factor
   Factor pd_country{ {COUNTRY}, {2}, { 5, 100 } };
   country.Normalize();
   // now change the factors for Season and Rain to have an additional dependency to country
   Factor cpd_season_given_country{ {SEASON,COUNTRY}, {2,2}, {0.8,0.2,0.2,0.8} };
   Factor cpd_rain_given_country_and_season{ {RAIN,SEASON,COUNTRY}, {2,2,2}, {0.82,0.18,0.86,0.14,0.4,0.6,0.6,0.4} };

```

So refining the model adds random variables and dependencies. It makes the model more accurate, but also more complex. Manual calculations with factor operations soon get tedious.

What we need are general algorithms, that can be applied to any Bayesian Network, without having to manually specify the factor operations to get a value of interest.

## Variable Elimination
Applying manual factor operations on Bayesian Networks is tedious, but can also be very inefficient. 
If you have a network with many variables, you could first calculate the joint distribution (which contains the knowledge about the whole system) and then marginalize out all variables execpt the variables you are interested in. If you would take a closer look at the operations done in the marginalization step, you would see that some terms would get calculated more than once.

VariableElimination is an algorithm for Bayesian Networks that uses Dynamic Programming to store intermediate terms until they are needed again, to avoid duplicated calculations.

Let's see how it works. VariableElimination takes in a list of factors (i.e. the Bayesian Network) and a list of variables to eliminate and returns a list of remaining factors after the elimination of the given variables. 

If we eliminate Country and Season, we get the marginal probability of Rain.

```
   std::vector <Factor> factors{
      pd_country,
      cpd_season_given_country,
      cpd_rain_given_country_and_season
   };

   auto f(factors);  // make copy to preserve original factors
   // eliminate Season and Country gives the marginal of Rain
   VariableElimination(f, { SEASON,COUNTRY });
   const auto m_rain = f.front();
```
If you want the probabilities of other variables, you have to run Variable Elimination again.
For example, to calculate the marginal probability of Season:
```
   f = factors;  // restore original factors
   // eliminate Rain and Country gives the marginal of Season
   VariableElimination(f, { RAIN,COUNTRY });
   const auto m_season = f.front();
   std::cout << "m_season:\n" << m_season;
```
For another example, see [Water Sprinkler](../examples/water_sprinkler/water_sprinkler_variable_elimination.cpp)

## Clique Tree (Junction Tree)

VariableElimination is apropriate, if you want to infer only one value of interest from the Bayes Net, because for every value, you have to run the whole inference.

If you want to infer multiple values of interest from the Bayes Net, there is a more efficient algorithm called **Clique Tree Algorithm**. 

The Clique Tree Algorithm takes in a Bayes Net and clusters the variables such that the structure after clustering forms a tree. Then inference can be done efficiently with a method called **Belief Propagation** on this tree structure.

**Cliques** are clusters of variables with a certain structure. In a clique every two distinct vertices are adjacent. Cliques have a potential given by a factor. For example, the factor F(Season, Country, Rain) is the potential of the clique with variables {Season, Country, Rain}. 

The algorithm has 3 steps:

1. **Creation**: create the clique tree and initialize its nodes to initial potentials
2. **Calibration**: the nodes exchange information with other nodes, aka. message passing
3. **Retrieval**: get the values of interest form containing cliques

Let's see the code for the steps:

1. **Creation**:
```
   CliqueTree c{ f, {} };
```
2. **Calibration**:
```
   c.Calibrate();
```
3. **Retrieval**:
```
   // get ordered list of all variables
   std::vector<uint32_t> vars{ UniqueVars(c.CliqueList()) };
   // calculate marginals for all variables
   std::vector<Factor> m(vars.size());
   // for all variables and cliques
   for (const auto& var : vars) {
      for (const auto& clique : c.CliqueList()) {
         // if variable is in scope of clique
         if (!Intersection({ var }, clique.Var()).values.empty()) {
            // marginalize all variables except current variable to get marginal        
            m[var] = clique.Marginalize(Difference(clique.Var(), { var }).values);
            m[var].Normalize();
         }
      }
   }
```
There is a convenience function to do all steps in one:
```
   m = CliqueTreeComputeExactMarginalsBP(f, {}, false);
```